In [4]:
import numpy as np    
import pandas as pd
import os
from tqdm import tqdm_notebook as tqdm

from skimage import data, io
from skimage.transform import resize

from sklearn.model_selection import train_test_split

In [57]:
train_features

,filepath,site
id,,
ZJ000000,../data/train_features/ZJ000000.jpg,S0120
ZJ000001,../data/train_features/ZJ000001.jpg,S0069
ZJ000002,../data/train_features/ZJ000002.jpg,S0009
ZJ000003,../data/train_features/ZJ000003.jpg,S0008
ZJ000004,../data/train_features/ZJ000004.jpg,S0036
...,...,...
ZJ016483,../data/train_features/ZJ016483.jpg,S0093
ZJ016484,../data/train_features/ZJ016484.jpg,S0043
ZJ016485,../data/train_features/ZJ016485.jpg,S0089


In [64]:
train_features = pd.read_csv("../data/train_features.csv", index_col="id")
train_labels = pd.read_csv("../data/train_labels.csv", index_col="id")

train_features["filepath"] = train_features["filepath"].apply(
    lambda x: os.path.join("/Users/nicholasabad/Desktop/workspace/driven-data-conservision/data", x)
)

In [ ]:
class DataLoader:
    def __init__(
        self, 
        training_features: pd.DataFrame,
        training_labels: pd.DataFrame,
        validation_split: float=0.2,
        resized_image_shape: tuple=(224, 224),
        random_state: int=42
    ):
        self.training_features = training_features
        self.training_labels = training_labels
        self.validation_split = validation_split
        self.resized_image_shape = resized_image_shape
        self.random_state = random_state

    def _convert_images_to_numpy_array_and_resize(
        self,
        paths_to_images: list,
    ):
        # Helper function to ensure all images are RGB
        def ensure_rgb(image):
            if len(image.shape) == 2:  # Grayscale (H, W)
                return np.stack([image] * 3, axis=-1)  # Convert to (H, W, 3)
            elif len(image.shape) == 3 and image.shape[2] == 3:  # Already RGB
                return image
            else:
                raise ValueError(f"Unexpected image shape: {image.shape}")
            
        return np.stack([
            ensure_rgb(resize(
                io.imread(path),
                output_shape=self.resized_image_shape,
                anti_aliasing=True,
            )) / 255.0
            for path in tqdm(paths_to_images, desc="Loading data into np array:")
        ], axis=0)
        
    def split_data(self):
        # Split the data into a training and validation split.
        y = self.training_labels
        x = self.training_features.loc[y.index].filepath.to_frame()
        
        x_train, x_val, y_train, y_val = train_test_split(
            x, y, stratify=y, test_size=self.validation_split, random_state=self.random_state
        )
        
        self.x_train = self._convert_images_to_numpy_array_and_resize(x_train["filepath"])
        self.x_val = self._convert_images_to_numpy_array_and_resize(x_val["filepath"])
        
        self.y_train = y_train
        self.y_val = y_val
        
        
    
        

In [71]:
data_loader = DataLoader(
    training_features = train_features,
    training_labels = train_labels,
    validation_split = 0.2,
    resized_image_shape = (224, 224),
    random_state = 42
)

data_loader.split_data()

/var/folders/2s/38kg81v526j7qcgd8gdq8jd00000gn/T/ipykernel_13587/2928171840.py:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for path in tqdm(paths_to_images, desc="Loading data into np array:")


Loading data into np array::   0%|          | 0/13190 [00:00<?, ?it/s]

: 

In [48]:
train_features

,filepath,site
id,,
ZJ000000,../data/train_features/ZJ000000.jpg,S0120
ZJ000001,../data/train_features/ZJ000001.jpg,S0069
ZJ000002,../data/train_features/ZJ000002.jpg,S0009
ZJ000003,../data/train_features/ZJ000003.jpg,S0008
ZJ000004,../data/train_features/ZJ000004.jpg,S0036
...,...,...
ZJ016483,../data/train_features/ZJ016483.jpg,S0093
ZJ016484,../data/train_features/ZJ016484.jpg,S0043
ZJ016485,../data/train_features/ZJ016485.jpg,S0089


In [13]:
x = train_features.loc[y.index].filepath.to_frame()
x

,filepath
id,
ZJ000625,../data/train_features/ZJ000625.jpg
ZJ008571,../data/train_features/ZJ008571.jpg
ZJ008106,../data/train_features/ZJ008106.jpg
ZJ011175,../data/train_features/ZJ011175.jpg
ZJ003323,../data/train_features/ZJ003323.jpg
...,...
ZJ000177,../data/train_features/ZJ000177.jpg
ZJ007834,../data/train_features/ZJ007834.jpg
ZJ015074,../data/train_features/ZJ015074.jpg


In [35]:
def _convert_images_to_numpy_array(
    paths_to_images: list,
):
    # Helper function to ensure all images are RGB
    def ensure_rgb(image):
        if len(image.shape) == 2:  # Grayscale (H, W)
            return np.stack([image] * 3, axis=-1)  # Convert to (H, W, 3)
        elif len(image.shape) == 3 and image.shape[2] == 3:  # Already RGB
            return image
        else:
            raise ValueError(f"Unexpected image shape: {image.shape}")
        
    return np.stack([
        ensure_rgb(resize(
            io.imread(path),
            output_shape=(224, 224),
            anti_aliasing=True,
        )) / 255.0
        for path in tqdm(paths_to_images, desc="Loading data into np array...")
    ], axis=0)